In [4]:
import pandas as pd
import numpy as np

<h2>DATA COLLECTION</h2>

In [5]:
df_course = pd.read_csv(r'C:\Users\aa22788\udemy course.csv',sep=';')
df_review = pd.read_csv(r'C:\Users\aa22788\udemy review.csv',sep=';')


In [6]:
df_course.head()

,course_id,course_name,subs,num_reviews,num_lectures,duration,last_update,publish_date,created_date,original_price,discount_price,level,label
0,8082,Ruby Programming for Beginners,28445,643,56,6.0,21/08/2019 00:00,08/07/2011 21:32,14/06/2011 16:09,1050000,182000.0,All Levels,Ruby
1,8203,Basic and Advanced HTML Lessons,1550,12,15,1.0,10/07/2017 00:00,20/06/2011 17:48,20/06/2011 19:59,350000,182000.0,All Levels,HTML
2,8319,Git Basics: In Theory and Practice,291,12,31,7.5,NaN,29/07/2011 20:24,28/06/2011 04:48,280000,182000.0,All Levels,Git
3,8324,Javascript for Beginners,3840,258,48,3.0,16/07/2019 00:00,09/07/2011 05:43,28/06/2011 08:26,280000,182000.0,All Levels,JavaScript
4,8325,HTML Tutorial: HTML & CSS for Beginners,12521,297,82,4.0,16/07/2019 00:00,09/09/2011 15:28,28/06/2011 08:27,280000,182000.0,All Levels,HTML


In [7]:
df_review.head()

,course_id,rating,review_content,date
0,8082,4.5,Delivers more than expected. Thank you!,29/07/2019 11:19
1,8082,2.5,This course would be a good choice if 1) you a...,22/06/2019 11:25
2,8082,5.0,I am new to Ruby but have programmed in Python...,14/01/2019 12:40
3,8082,4.0,The instructor has provided quite a lot of inf...,04/12/2018 08:03
4,8082,5.0,"Hare Kṛṣṇa, Hare Kṛṣṇa, Kṛṣṇa Kṛṣṇa, Hare Hare...",16/11/2018 19:12


In [ ]:
df

<h2>DATA CLEANING</h2>

1. Names are descriptive enough
2. No duplicates in course dataset, 1 duplicate in review dataset
3. Convert all date columns from object to datetime
4. create publish year from published date

In [34]:
#check datatypes
df_course.dtypes

course_id                 object
course_name               object
subs                       int64
num_reviews                int64
num_lectures               int64
duration                 float64
last_update       datetime64[ns]
publish_date      datetime64[ns]
created_date      datetime64[ns]
original_price             int64
discount_price           float64
level                     object
label                     object
dtype: object

In [26]:
df_review.dtypes

course_id           int64
rating            float64
review_content     object
date               object
dtype: object

convert all dates column datatype from object to datetime, and course_id to object

In [32]:
df_course.head(1)

,course_id,course_name,subs,num_reviews,num_lectures,duration,last_update,publish_date,created_date,original_price,discount_price,level,label
0,8082,Ruby Programming for Beginners,28445,643,56,6.0,2019-08-21,2011-08-07 21:32:00,2011-06-14 16:09:00,1050000,182000.0,All Levels,Ruby


In [31]:
#convert date column to datetime
df_course['last_update'] = pd.to_datetime(df_course['last_update'])
df_course['publish_date'] = pd.to_datetime(df_course['publish_date'])
df_course['created_date'] = pd.to_datetime(df_course['created_date'])
df_review['date'] = pd.to_datetime(df_review['date'])


In [33]:
#convert course id from int to object. 
df_course['course_id'] = df_course['course_id'].apply(str)

In [18]:
df_course.duplicated().sum()

0

In [23]:
# 1 duplicate
df_review.duplicated().sum()

0

In [22]:
#drop the duplicates
df_review.drop_duplicates(inplace=True)

In [10]:
df_course.shape

(9243, 13)

In [9]:
df_review.shape

(488758, 4)

In [11]:
df_course.sample(6)

,course_id,course_name,subs,num_reviews,num_lectures,duration,last_update,publish_date,created_date,original_price,discount_price,level,label
6686,1741694,Chatbot Boom -Learn Facebook Messenger Marketi...,282,7,19,1.5,23/09/2018 00:00,18/09/2018 01:25,10/06/2018 20:22,2800000,182000.0,All Levels,Chatbot
1318,533396,A Complete Introduction to the C++ Programming...,1189,90,67,3.0,09/09/2015 00:00,09/09/2015 17:49,21/06/2015 10:11,700000,182000.0,All Levels,C++
2617,893616,"Latest CMS Blog,Freelancing Services Template ...",251,33,302,55.5,20/12/2017 00:00,04/07/2016 22:11,01/07/2016 12:14,700000,182000.0,All Levels,PHP
8389,2231162,Spring Boot: Learn Spring Boot From Scratch,103,20,43,4.0,27/03/2019 00:00,29/03/2019 19:41,20/02/2019 15:28,2800000,182000.0,All Levels,Spring Boot
135,49340,The Ultimate Java 9 Tutorial - From beginner t...,5008,92,236,25.0,09/01/2018 00:00,02/05/2013 09:18,11/04/2013 08:12,280000,182000.0,All Levels,Java
6107,1615936,Master the JavaScript Interview,208,35,37,3.5,26/11/2018 00:00,09/05/2018 20:26,25/03/2018 22:21,630000,182000.0,All Levels,JavaScript


There are non english stuff. We'll revisit 

In [16]:
df_review.sample(6)

,course_id,rating,review_content,date
69395,375054,1.0,This course was pulled from a webinar. There ...,18/03/2016 12:13
474134,2056529,4.0,Slow and steady. Great job with explaining ste...,15/01/2019 21:58
366373,1329100,4.0,Bonne présentation!,12/09/2017 07:58
280625,1039062,5.0,good stuff,28/11/2017 11:19
125151,592594,1.5,good content but i would expect way more for t...,15/03/2018 10:30
371651,1344518,5.0,enjoy in course,14/09/2017 01:44


In [24]:
df_course.head(2)

,course_id,course_name,subs,num_reviews,num_lectures,duration,last_update,publish_date,created_date,original_price,discount_price,level,label
0,8082,Ruby Programming for Beginners,28445,643,56,6.0,21/08/2019 00:00,08/07/2011 21:32,14/06/2011 16:09,1050000,182000.0,All Levels,Ruby
1,8203,Basic and Advanced HTML Lessons,1550,12,15,1.0,10/07/2017 00:00,20/06/2011 17:48,20/06/2011 19:59,350000,182000.0,All Levels,HTML


<h2>DATA ANALYSIS</h2>

1. Courses with the highest number of reviews
2. Do higher ratings correlate with: course duration, number of lectures, original_price, discount_price, level?
3. Correlation between frequently updated courses and average ratings
4. words associated with good reviews

 wHAT TYPE OF RECOMMENDER SYSTEM DO WE USE?

Most popular courses reviewed

In [38]:
df_courseCopy = df_course.copy()
df_courseCopy.sort_values(by='subs',ascending = False,inplace=True )
df_courseCopy.reset_index(inplace=True,drop =True)
df_courseCopy.head()

,course_id,course_name,subs,num_reviews,num_lectures,duration,last_update,publish_date,created_date,original_price,discount_price,level,label
0,567828,Complete Python Bootcamp: Go from zero to hero...,685057,186085,186,24.0,2019-04-09,2015-12-10 21:42:00,2015-07-29 00:12:00,2730000,182000.0,All Levels,Python
1,625204,The Web Developer Bootcamp,460682,135796,399,47.0,2019-05-09,2015-02-11 21:13:00,2015-09-28 21:32:00,2800000,182000.0,All Levels,Web Development
2,533682,Java Programming Masterclass for Software Deve...,357090,83454,392,79.0,2019-08-22,2015-07-21 22:01:00,2015-06-21 20:25:00,2100000,182000.0,All Levels,Java
3,41295,Learn HTML5 Programming From Scratch,316033,13294,46,10.5,2018-11-26,2013-02-14 07:03:00,2013-12-02 09:25:00,280000,182000.0,All Levels,HTML5
4,258316,Complete C# Unity Developer 2D: Learn to Code ...,315197,63783,221,35.0,2019-09-14,2014-03-09 19:30:00,2014-09-07 19:17:00,2730000,182000.0,All Levels,Unity


Most popular 'label' according to number of subscribers

In [51]:
df_courseCopy = df_course.copy()
df_labelCat = df_courseCopy.groupby(['label'],as_index=False)['subs'].sum()
# df_labelCat.to_frame()
df_labelCat.reset_index(level=0, drop=True,inplACE=True)
# df_labelCat = df_labelCat.to_frame()
# df_labelCat.sort_values(by='subs', ascending = False, inplace=True)
# df_labelCat.reset_index(inplace=True)
# # df_courseCopy.sort_values(by='num_reviews',ascending = False,inplace=True )
# # df_courseCopy.reset_index(inplace=True,drop =True)
# # df_courseCopy.head()
type(df_labelCat)

AttributeError: 'SeriesGroupBy' object has no attribute 'reset_index'

we use label instead of  'category' because users are quite specific on the topic they'd like to learn


In [ ]:
#words

In [ ]:
df_course['publish_date']

<h2>RECOMMENDATION SYSTEM</h2>
How to build a recommendation system via title and review sentiment using similarity. 